## 2D Transformation

In [3]:
import matplotlib.pyplot as plt
import numpy as np

# Points a, b, c, d
a, b, c, d = [0, 0, 1], [0, 1, 1], [1, 1, 1], [1, 0, 1]

X = np.array([a, b, c, d]).T
# Y = X + np.array([3, 4, 1]).reshape(3,1) # transfromed version of X

# theta = np.pi/180*60
t = np.array([2, 1]).T
# H = np.array([[np.cos(theta), -np.sin(theta), t[0]], [np.sin(theta), np.cos(theta), t[1]], [0, 0, 1]])
# Y = H @ X

A  = np.array([[1, 0.5, t[0]], [2, 1.5, t[1]], [0, 0, 1]]) # Affine transformaiton
Y = A @ X
# Homographic transformation

x = np.append(X[0, :], X[0, 0])
y = np.append(X[1, :], X[1, 0])
fig, ax = plt.subplots(1,1)
ax.plot(x, y, color='g')

x = np.append(Y[0, :], Y[0, 0])
y = np.append(Y[1, :], Y[1, 0])
ax.plot(x, y, color='r')
ax.set_aspect('equal')
plt.show()

ImportError: DLL load failed while importing _c_internal_utils: The specified module could not be found.

## Transforming using a given Homography

In [ ]:
# Transfroming the first image onto the second
# Gaffiti images: https://www.robots.ox.ac.uk/~vgg/data/affine/
import numpy as np
import cv2 as cv
im1 = cv.imread(r'./images/graf/img1.ppm', cv.IMREAD_ANYCOLOR)
im2 = cv.imread(r'./images/graf/img2.ppm', cv.IMREAD_ANYCOLOR)
assert im1 is not None
assert im2 is not None

# H = np.array([[8.7976964e-01,   3.1245438e-01,  -3.9430589e+01],
#   [-1.8389418e-01,   9.3847198e-01,   1.5315784e+02],
#    [1.9641425e-04,  -1.6015275e-05,   1.0000000e+00]])

with open(r'./images/graf/H1to2p') as f:
   H = np.array([[float(h) for h in line.split()] for line in f])

im1to2 = cv.warpPerspective(im1, H, (1000,1000))

cv.namedWindow('Image 1', cv.WINDOW_AUTOSIZE)
cv.imshow('Image 1', im1)
cv.waitKey(0)
cv.namedWindow('Image 2', cv.WINDOW_AUTOSIZE)
cv.imshow('Image 2', im2)
cv.waitKey(0)
cv.namedWindow('Image 1 Warped', cv.WINDOW_AUTOSIZE)
cv.imshow('Image 1 Warped', im1to2)
cv.waitKey(0)
cv.destroyAllWindows()

## Transforming the Second Image onto the First

In [ ]:
import numpy as np
import cv2 as cv
im1 = cv.imread(r'./images/graf/img1.ppm', cv.IMREAD_ANYCOLOR)
im2 = cv.imread(r'./images/graf/img2.ppm', cv.IMREAD_ANYCOLOR)
assert im1 is not None
assert im2 is not None

with open(r'./images/graf/H1to2p') as f:
   H = np.array([[float(h) for h in line.split()] for line in f])

im2to1 = cv.warpPerspective(im2, np.linalg.inv(H), (1000,1000))
im2to1[0:im1.shape[0], 0:im1.shape[1]] = im1
im2to1[0:im1.shape[0], 0:im1.shape[1]] += np.array([50, 0, 0], dtype=np.uint8) # Change te color a little

cv.namedWindow('Image 1', cv.WINDOW_AUTOSIZE)
cv.imshow('Image 1', im1)
cv.waitKey(0)
cv.namedWindow('Image 2', cv.WINDOW_AUTOSIZE)
cv.imshow('Image 2', im2)
cv.waitKey(0)
cv.namedWindow('Image 1 Warped', cv.WINDOW_AUTOSIZE)
cv.imshow('Image 1 Warped', im2to1)
cv.waitKey(0)
cv.destroyAllWindows()

## Computing the Homography using Mouse-Clicked Points and Warping

In [ ]:
import cv2 as cv
import numpy as np


N = 5
global n 
n = 0    
p1 = np.empty((N,2))
p2 = np.empty((N,2))

# mouse callback function
def draw_circle(event,x,y,flags,param):
    global n
    p = param[0]
    if event == cv.EVENT_LBUTTONDOWN:
        cv.circle(param[1],(x,y),5,(255,0,0),-1)        
        p[n] = (x,y)
        n += 1
          

im1 = cv.imread('./images/graf/img1.ppm', cv.IMREAD_ANYCOLOR)
im4 = cv.imread('./images/graf/img4.ppm', cv.IMREAD_ANYCOLOR)

im1copy = im1.copy()
im4copy = im4.copy()

cv.namedWindow('Image 1', cv.WINDOW_AUTOSIZE)

param = [p1, im1copy]
cv.setMouseCallback('Image 1',draw_circle, param)

while(1):
    cv.imshow("Image 1", im1copy)
    if n == N:
        break
    if cv.waitKey(20) & 0xFF == 27:
        break


param = [p2, im4copy]
n = 0
cv.namedWindow("Image 4", cv.WINDOW_AUTOSIZE)
cv.setMouseCallback('Image 4',draw_circle, param)

while(1):
    cv.imshow("Image 4", im4copy)
    if n == N:
        break
    if cv.waitKey(20) & 0xFF == 27:
        break


print(p1)
print(p2)

H = cv.findHomography(p1, p2)
H = H[0]
print(H)
im4_warped = cv.warpPerspective(im4, np.linalg.inv(H), (1000,1000))
im4_warped[0:im1.shape[0], 0:im1.shape[1]] = im1

cv.namedWindow("Image 4 Warped", cv.WINDOW_AUTOSIZE)
cv.imshow("Image 4 Warped", im4_warped)
cv.waitKey(0)
cv.destroyAllWindows()

## Computing the Homogrpahy Using Mouse-Clicked Points without OpenCV

In [ ]:
import cv2 as cv
import numpy as np


N = 5
global n 
n = 0    
p1 = np.empty((N,2))
p2 = np.empty((N,2))

# mouse callback function
def draw_circle(event,x,y,flags,param):
    global n
    p = param[0]
    if event == cv.EVENT_LBUTTONDOWN:
        cv.circle(param[1],(x,y),5,(255,0,0),-1)        
        p[n] = (x,y)
        n += 1
          

im1 = cv.imread('./images/graf/img1.ppm', cv.IMREAD_ANYCOLOR)
im4 = cv.imread('./images/graf/img4.ppm', cv.IMREAD_ANYCOLOR)

im1copy = im1.copy()
im4copy = im4.copy()
   

cv.namedWindow('Image 1', cv.WINDOW_AUTOSIZE)

param = [p1, im1copy]
cv.setMouseCallback('Image 1',draw_circle, param)

while(1):
    cv.imshow("Image 1", im1copy)
    if n == N:
        break
    if cv.waitKey(20) & 0xFF == 27:
        break


param = [p2, im4copy]
n = 0
cv.namedWindow("Image 4", cv.WINDOW_AUTOSIZE)
cv.setMouseCallback('Image 4',draw_circle, param)

while(1):
    cv.imshow("Image 4", im4copy)
    if n == N:
        break
    if cv.waitKey(20) & 0xFF == 27:
        break


print(p1)
print(p2)

A = np.empty((2*N, 9))

for i in range(0,N):
    xt = p1[i, :]
    xpt = p2[i, :]
    A[2*i, :] = np.array([0, 0, 0, xt[0], xt[1], 1, -xpt[1]*xt[0], -xpt[1]*xt[1], -xpt[1]])
    A[2*i+1, :] = np.array([xt[0], xt[1], 1, 0, 0, 0, -xpt[0]*xt[0], -xpt[0]*xt[1], -xpt[0]])    
    
print(A)

_, S, Vh = np.linalg.svd(np.matmul(A.T, A)) # See https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html
print(S)
Hhat = Vh[-1, :]
Hhat = Hhat.reshape(3,3)
Hhat = Hhat/Hhat[2,2]
print('Hhat', Hhat)

H = cv.findHomography(p1, p2)
H = H[0]
print('H using find homography', H)

im4_warped = cv.warpPerspective(im4, np.linalg.inv(Hhat), (1000,1000))
im4_warped[0:im1.shape[0], 0:im1.shape[1]] = im1

cv.namedWindow("Image 4 Warped", cv.WINDOW_AUTOSIZE)
cv.imshow("Image 4 Warped", im4_warped)
cv.waitKey(0)
cv.destroyAllWindows()